<a href="https://colab.research.google.com/github/Ibraheem101/Data-Science-learning/blob/main/huggingface/2_hf_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tokenizers

### Loading and Saving

In [1]:
!pip install transformers[sentencepiece] # dev version

In [2]:
import torch

In [3]:
from transformers import BertTokenizer

Loading the BERT tokenizer trained with the same checkpoint as BERT is done the same way as loading the model, except we use the BertTokenizer class

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [7]:
tokenizer("I'm still learning how to use transformers")

{'input_ids': [101, 146, 112, 182, 1253, 3776, 1293, 1106, 1329, 11303, 1468, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [8]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

### Encoding
Translating text to numbers is known as encoding. Encoding is done in a two-step process: the tokenization, followed by the conversion to input ID or Numericalization.

* The first step is to split the text into words (or parts of words, punctuation symbols, etc.), usually called tokens. There are multiple rules that can govern that process, which is why we need to instantiate the tokenizer using the name of the model, to make sure we use the same rules that were used when the model was pretrained

* The second step is to convert those tokens into numbers, so we can build a tensor out of them and feed them to the model. To do this, the tokenizer has a vocabulary, which is the part we download when we instantiate it with the from_pretrained() method. Again, we need to use the same vocabulary used when the model was pretrained.

In [9]:
sequence = "I'm still learning how to use transformers"

In [10]:
tokens = tokenizer.tokenize(sequence)

In [11]:
print(tokens)

['I', "'", 'm', 'still', 'learning', 'how', 'to', 'use', 'transform', '##ers']


In [12]:
ids = tokenizer.convert_tokens_to_ids(tokens)

In [13]:
print(ids)

[146, 112, 182, 1253, 3776, 1293, 1106, 1329, 11303, 1468]


In [14]:
torch.tensor(ids)

tensor([  146,   112,   182,  1253,  3776,  1293,  1106,  1329, 11303,  1468])

### Decoding
Decoding is going the other way around: from vocabulary indices, we want to get a string.

In [15]:
decoded_string = tokenizer.decode([146, 112, 182, 1253, 3776, 1293, 1106, 1329, 11303, 1468])

In [16]:
print(decoded_string)

I'm still learning how to use transformers


### Handling multiple sequences


In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [18]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sentence2 = "This is the best course in the world!"
tokens = tokenizer.tokenize(sentence2)
ids = tokenizer.convert_tokens_to_ids(tokens)

In [19]:
input_ids = torch.tensor(ids)
input_ids

tensor([2023, 2003, 1996, 2190, 2607, 1999, 1996, 2088,  999])

In [21]:
model(input_ids) # Will fail

IndexError: ignored

The problem is that we sent a single sequence to the model, whereas 🤗 Transformers models expect multiple sentences by default. Here we tried to do everything the tokenizer did behind the scenes when we applied it to a sequence. But if you look closely, you’ll see that the tokenizer didn’t just convert the list of input IDs into a tensor, it added a dimension on top of it:

In [22]:
tokenizer(sentence2, return_tensors="pt")['input_ids']

tensor([[ 101, 2023, 2003, 1996, 2190, 2607, 1999, 1996, 2088,  999,  102]])

Let's try this again but we'll add a new dimension

In [23]:
input_ids = torch.tensor([ids])
input_ids

tensor([[2023, 2003, 1996, 2190, 2607, 1999, 1996, 2088,  999]])

In [24]:
model(input_ids), model(input_ids).logits # Will run

(SequenceClassifierOutput(loss=None, logits=tensor([[-4.1817,  4.5285]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None),
 tensor([[-4.1817,  4.5285]], grad_fn=<AddmmBackward0>))

Batching is the act of sending multiple sentences through the model, all at once. If you only have one sentence, you can just build a batch with a single sequence:

In [25]:
batched_ids = [ids, ids]

In [26]:
input_batch_ids = torch.tensor(batched_ids)
input_batch_ids

tensor([[2023, 2003, 1996, 2190, 2607, 1999, 1996, 2088,  999],
        [2023, 2003, 1996, 2190, 2607, 1999, 1996, 2088,  999]])

In [27]:
model(input_batch_ids)

SequenceClassifierOutput(loss=None, logits=tensor([[-4.1817,  4.5285],
        [-4.1817,  4.5285]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Batching allows the model to work when you feed it multiple sentences. Using multiple sequences is just as simple as building a batch with a single sequence. There’s a second issue, though. When you’re trying to batch together two (or more) sentences, they might be of different lengths. If you’ve ever worked with tensors before, you know that they need to be of rectangular shape, so you won’t be able to convert the list of input IDs into a tensor directly. To work around this problem, we usually pad the inputs.

### Padding the inputs


In [28]:
# Cannot convert to tensor
try:
    torch.tensor([
        [200, 200, 200],
        [200, 200]
    ])
except ValueError:
    print("ValueError: expected sequence of length l at dim d (got m)")

ValueError: expected sequence of length l at dim d (got m)


In [29]:
# Padding
padding_id = 100
torch.tensor([
        [200, 200, 200],
        [200, 200, padding_id]
])

tensor([[200, 200, 200],
        [200, 200, 100]])

In [30]:
sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


There’s something wrong with the logits in our batched predictions: the second row should be the same as the logits for the second sentence, but we’ve got completely different values!

This is because the key feature of Transformer models is attention layers that contextualize each token. These will take into account the padding tokens since they attend to all of the tokens of a sequence. To get the same result when passing individual sentences of different lengths through the model or when passing a batch with the same sentences and padding applied, we need to tell those attention layers to ignore the padding tokens. This is done by using an attention mask.

Just like above: We strongly recommend passing in an `attention_mask` since your input_ids may be padded

### Attention Masks

Attention masks are tensors with the exact same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to (i.e., they should be ignored by the attention layers of the model).

In [31]:
sequence1_ids = torch.tensor([[200, 200, 200]])
sequence2_ids = torch.tensor([[200, 200]])
batched_ids = torch.tensor([
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
])

attention_mask = torch.tensor([[1, 1, 1],
                               [1, 1, 0]])

In [32]:
print(model(batched_ids, attention_mask=attention_mask))

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


We can see that the logits are similar to those of sequence2

### Using longer sequences

Models have different supported sequence lengths, and some specialize in handling very long sequences. Longformer is one example, and another is LED. If you’re working on a task that requires very long sequences, we recommend you take a look at those models.

Otherwise, we recommend you truncate your sequences by specifying the max_sequence_length parameter:

`sequence = sequence[:max_sequence_length]`

# Putting it all together

### Wrapping up: From tokenizer to model
Now that we’ve seen all the individual steps the tokenizer object uses when applied on texts, let’s see one final time how it can handle multiple sequences (padding!), very long sequences (truncation!), and multiple types of tensors with its main API:

In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = ["Models have different supported sequence lengths, and some specialize in handling very long sequences", "Putting it all together"]
tokens = tokenizer(sequence, padding=True, truncation=True, return_tensors="pt")

output = model(**tokens)

In [41]:
torch.nn.functional.softmax(output.logits, dim=-1)

tensor([[0.7964, 0.2036],
        [0.0087, 0.9913]], grad_fn=<SoftmaxBackward0>)

In [36]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}